2025-05-29 01:31:11.335 | INFO     | local_interpreter:execute_code:61 - 执行代码: print('hello world')
2025-05-29 01:33:21.840 | INFO     | local_interpreter:initialize:33 - 初始化本地内核
2025-05-29 01:33:22.640 | INFO     | local_interpreter:execute_code_:141 - 执行代码: import os
work_dir = r'./work'
os.makedirs(work_dir, exist_ok=True)
os.chdir(work_dir)
print('当前工作目录:', os.getcwd())
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'Microsoft YaHei', 'WenQuanYi Micro Hei', 'PingFang SC', 'Hiragino Sans GB', 'Heiti SC', 'DejaVu Sans', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.size'] = 12
mpl.rcParams['axes.labelsize'] = 12
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10

2025-05-29 01:35:19.832 | INFO     | local_interpreter:execute_code:61 - 执行代码: print('hello world')
2025-05-29 01:35:19.836 | INFO     | local_interpreter:

In [3]:
code_interpreter.initialize()

In [7]:
a= "print('hello world')"
# text_to_gpt, error_occurred,error_message
(a,b,c)=code_interpreter.execute_code(a)


有代码运行环境，那么我可以直接用以下流程：
增加一个字典，使用导入写


# 开源代码

### 第一步planner

In [4]:
from planner.tools import get_pla_user
from planner.prompt import get_planer
from utils.api import gpt_chat
type="opt"
question="./test_case/o4/question.txt"

planer = get_planer(problem_type=type)
info = get_pla_user(ques=question,problem_type=type)

# response1 = gpt_chat(sys=planer,user=info,provider="deepseek")


## 匹配函数

In [2]:
from planner.tools import get_plan
from utils.rag import ChoromaDBManager

plan = get_plan(str_path="./test_case/o4/plan_deepseek.txt")
chroma_db = ChoromaDBManager("./test_case/o4/tool_db")

/home/zyx/anaconda3/envs/base1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
chroma_db.store_tools_to_db(dir_path="./tool_doc_md")

✅ 已存储: statistic.md (包含 2 个工具块)
✅ 已存储: data_clean.md (包含 5 个工具块)
✅ 已存储: graph_optimization.md (包含 5 个工具块)
✅ 已存储: machine_learning.md (包含 15 个工具块)
✅ 已存储: math_optimization.md (包含 10 个工具块)
✅ 已存储: feature_process.md (包含 7 个工具块)
✅ 已存储: evaluate_model.md (包含 3 个工具块)


In [ ]:
funcs=chroma_db.get_all_tools(plan)

[{'tool_name': 'solve_lp',
  'content': '**Name:** solve_lp  \n**Description:** 用于简单线性规划问题，线性规划求解函数。\n**Applicable Situations:**  \n- 需要求解目标函数最小化的线性规划问题  \n\n**Parameters:**  \n- `c`:  \n  - \u200b**Type:** `array-like`  \n  - \u200b**Description:** 目标函数系数向量，维度需与变量数一致  \n\n- `A_ub`:  \n  - \u200b**Type:** `2D array-like | None`  \n  - \u200b**Description:** 不等式约束系数矩阵（默认None），每行对应一个约束条件  \n\n- `b_ub`:  \n  - \u200b**Type:** `array-like | None`  \n  - \u200b**Description:** 不等式约束右侧常数向量（默认None）  \n\n- `A_eq`:  \n  - \u200b**Type:** `2D array-like | None`  \n  - \u200b**Description:** 等式约束系数矩阵（默认None）  \n\n- `b_eq`:  \n  - \u200b**Type:** `array-like | None`  \n  - \u200b**Description:** 等式约束右侧常数向量（默认None）  \n\n- `bounds`:  \n  - \u200b**Type:** `list of tuples | None`  \n  - \u200b**Description:** 变量边界列表（默认None），每个元组表示变量下界和上界  \n  - \u200b**Example:** `[(0, None), (0, 5)]` 表示x₁≥0，x₂∈[0,5]  \n\n**Result:**  \n- 成功时返回包含最优解x和最优值fun的元组  \n- 失败时返回包含错误信息的字符串  \n\n**Example Call:**  \n```python\

## Developer

In [ ]:
from developer.prompt import get_developer
from developer.tools import get_dev_user
funcs = [{'tool_name': 'solve_lp',
  'content': '**Name:** solve_lp  \n**Description:** 用于简单线性规划问题，线性规划求解函数。\n**Applicable Situations:**  \n- 需要求解目标函数最小化的线性规划问题  \n\n**Parameters:**  \n- `c`:  \n  - \u200b**Type:** `array-like`  \n  - \u200b**Description:** 目标函数系数向量，维度需与变量数一致  \n\n- `A_ub`:  \n  - \u200b**Type:** `2D array-like | None`  \n  - \u200b**Description:** 不等式约束系数矩阵（默认None），每行对应一个约束条件  \n\n- `b_ub`:  \n  - \u200b**Type:** `array-like | None`  \n  - \u200b**Description:** 不等式约束右侧常数向量（默认None）  \n\n- `A_eq`:  \n  - \u200b**Type:** `2D array-like | None`  \n  - \u200b**Description:** 等式约束系数矩阵（默认None）  \n\n- `b_eq`:  \n  - \u200b**Type:** `array-like | None`  \n  - \u200b**Description:** 等式约束右侧常数向量（默认None）  \n\n- `bounds`:  \n  - \u200b**Type:** `list of tuples | None`  \n  - \u200b**Description:** 变量边界列表（默认None），每个元组表示变量下界和上界  \n  - \u200b**Example:** `[(0, None), (0, 5)]` 表示x₁≥0，x₂∈[0,5]  \n\n**Result:**  \n- 成功时返回包含最优解x和最优值fun的元组  \n- 失败时返回包含错误信息的字符串  \n\n**Example Call:**  \n```python\n# 求解 min -x1 + 4x2 \n# s.t.  x1 + x2 <= 5\n#       2x1 + x2 = 4\n#       x1 ∈ [0,3], x2 ≥ 0\nx_opt, f_opt = solve_lp(\n    c=[-1, 4],\n    A_ub=[[1, 1]],\n    b_ub=[5],\n    A_eq=[[2, 1]],\n    b_eq=[4],\n    bounds=[(0, 3), (0, None)]\n)\n```',
  'source_file': 'math_optimization.md',
  'distance': 0.5363641381263733}]
devloper_prompt = get_developer(problem_type=type,func=funcs)
user2 = get_dev_user(question=question,problem_type=type)

# response2 = gpt_chat(sys=devloper_prompt,user=user2)

## 编译

In [8]:
from utils.notebook_serializer import NotebookSerializer
from utils.local_interpreter import LocalCodeInterpreter
import sys
import json
import os
dirname = "./"
notebook = NotebookSerializer(dirname)

# agents code执行
code_interpreter = LocalCodeInterpreter(work_dir=dirname,notebook_serializer=notebook,task_id="111")
code_interpreter.initialize()